In [9]:
import re 

#double damma, double fatha, double kasera, damma, fatha, kasera, sukoon, shadd
TASHKEEL_SET = {'ٌ', 'ً', 'ٍ', 'ُ', 'َ', 'ِ', 'ْ', 'ٌّ', 'ّ'}
DIACRITICS_REGEX = re.compile('|'.join(TASHKEEL_SET))
def remove_tashkeel(data):
    return re.sub(DIACRITICS_REGEX, '', data)

print(remove_tashkeel('الْأَلْبَاب'))

الألباب


In [10]:
def one_hot_encode(indices, size):
    return [[1 if i == elem else 0 for i in range(size)] for elem in indices]

print(one_hot_encode([0, 1, 2],10) )

[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]]
